In [ ]:
!pip install requests parsel playwright

In [1]:
# Read modules 
from bs4 import BeautifulSoup
import requests, lxml, os, json
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import date 

## 1. Google scholar 

In [5]:
from parsel import Selector
import requests, re, json

def scrape_all_profiles_from_university(university_name: str):

    # https://docs.python-requests.org/en/master/user/quickstart/#passing-parameters-in-urls
    params = {
        "view_op": "search_authors",  # author results
        "mauthors": university_name,  # search query
        "hl": "en",                   # language
        "astart": 0                   # page number
    }

    # https://docs.python-requests.org/en/master/user/quickstart/#custom-headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.87 Safari/537.36",
    }

    profile_results = []

    profiles_is_present = True
    while profiles_is_present:

        html = requests.get("https://scholar.google.com/citations", params=params, headers=headers, timeout=30)
        select = Selector(html.text)

        # print(f"extracting authors at page #{params['astart']}.")

        for profile in select.css(".gs_ai_chpr"):
            name = profile.css(".gs_ai_name a::text").get()
            link = f'https://scholar.google.com{profile.css(".gs_ai_name a::attr(href)").get()}'
            
            affiliations = profile.css(".gs_ai_aff").xpath('normalize-space()').get()
            email = profile.css(".gs_ai_eml::text").get()
            cited_by = re.search(r"\d+", profile.xpath('//div[@class="gs_ai_cby"]').get()).group()  # Cited by 17143 -> 17143
            interests = profile.css(".gs_ai_one_int::text").getall()

            profile_results.append({
                "profile_name": name,
                "profile_link": link,
                "profile_affiliations": affiliations,
                "profile_email": email,
                "profile_city_by_count": cited_by,
                "profile_interests": interests
            })

        # if next page token is present -> update next page token and increment 10 to get the next page
        if select.css("button.gs_btnPR::attr(onclick)").get():
            # https://regex101.com/r/e0mq0C/1
            params["after_author"] = re.search(r"after_author\\x3d(.*)\\x26", select.css("button.gs_btnPR::attr(onclick)").get()).group(1)  # -> XB0HAMS9__8J
            params["astart"] += 10
        else:
            profiles_is_present = False
            
    return profile_results


# print(json.dumps(scrape_all_profiles_from_university(university_name="EM Normandie"), indent=2, ensure_ascii=False))
data = pd.DataFrame(scrape_all_profiles_from_university(university_name="EM Normandie"))
data['author_ID'] = data['profile_link'].apply(lambda x: x.split('=')[-1])
data.drop(['profile_affiliations', 'profile_email', 'profile_city_by_count'],axis =1, inplace = True)
data.head(3)

,profile_name,profile_link,profile_interests,author_ID
0,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ
1,Christophe Rosenberger,https://scholar.google.com/citations?hl=en&use...,"[Biometrics, Security & Privacy]",TpTmMrkAAAAJ
2,Jean-Christophe Avice,https://scholar.google.com/citations?hl=en&use...,"[Nitrogen and Sulfur Use Efficiency, Nutrients...",TkTj0I4AAAAJ


In [6]:
## Take the names from google scholar 
# import requests, lxml, os

headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

## get the data for each author 
citations_al = []; citations_sinc = []; h_index_al = []; h_index_sinc= []; i10_index_al = []; i10_index_sinc = []
for link in data['profile_link']:
    html = requests.get(link, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    # print(f'Collecting data for: {name}')
    for cited_by_public_access in soup.select('.gsc_rsb'):
        try:
            citations_all = cited_by_public_access.select_one('tr:nth-child(1) .gsc_rsb_sc1+ .gsc_rsb_std').text
        except:
            None  
        try:    
            citations_since = cited_by_public_access.select_one('tr:nth-child(1) .gsc_rsb_std+ .gsc_rsb_std').text
        except:
            None    
        try:
            h_index_all = cited_by_public_access.select_one('tr:nth-child(2) .gsc_rsb_sc1+ .gsc_rsb_std').text
        except:
            None 
        try:
            h_index_since = cited_by_public_access.select_one('tr:nth-child(2) .gsc_rsb_std+ .gsc_rsb_std').text
        except:
            None 
        try: 
            i10_index_all = cited_by_public_access.select_one('tr~ tr+ tr .gsc_rsb_sc1+ .gsc_rsb_std').text
        except: 
            None 
        try: 
            i10_index_since = cited_by_public_access.select_one('tr~ tr+ tr .gsc_rsb_std+ .gsc_rsb_std').text
        except: 
            None 
        # append the output into a list 
        citations_al.append(citations_all)
        citations_sinc.append(citations_since)
        h_index_al.append(h_index_all)
        h_index_sinc.append(h_index_since)
        i10_index_al.append(i10_index_all)
        i10_index_sinc.append(i10_index_since)
    # print(f'{citations_all}\n{citations_since2016}\n{h_index_all}\n{h_index_2016}\n{i10_index_all}\n{i10_index_2016}')
extra_data = pd.DataFrame([citations_al, citations_sinc, h_index_al, h_index_sinc, i10_index_al, i10_index_sinc]).T
extra_data.columns = ['Total citations', 'Citation l 5 yrs', 'Total H-index', "H-index l 5 yrs", "Total i10-index", 'i10-index l 5 yrs']
output = pd.concat([data, extra_data], axis =1)

In [7]:
output.sample(11)

,profile_name,profile_link,profile_interests,author_ID,Total citations,Citation l 5 yrs,Total H-index,H-index l 5 yrs,Total i10-index,i10-index l 5 yrs
45,"Josip Marić, PhD",https://scholar.google.com/citations?hl=en&use...,"[Digital technologies, Digital transformation,...",myltOpUAAAAJ,116,115,5,5,4,4
18,Julian Hofmann,https://scholar.google.com/citations?hl=en&use...,[],4ZCoTcUAAAAJ,528,349,7,7,6,6
35,Jean Pralong,https://scholar.google.com/citations?hl=en&use...,"[Carrières, Ressources Humaines, Schémas cogni...",YseQa5wAAAAJ,244,111,8,6,8,3
60,Frédéric caumon,https://scholar.google.com/citations?hl=en&use...,[Mathematical Finance],D0slPrsAAAAJ,10,3,1,1,1,0
6,Pr. Dr. Anis Jarboui,https://scholar.google.com/citations?hl=en&use...,[FINANCE et comptabilité],TSXET9gAAAAJ,2589,2337,28,26,69,65
43,Sina Nasiri,https://scholar.google.com/citations?hl=en&use...,"[Supply Chain Management, E-Commerce, Logistics]",RfEoWAgAAAAJ,151,105,2,2,1,1
0,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102
17,Pascal Hilber,https://scholar.google.com/citations?hl=en&use...,[Behavioral neurosciences],rN_7AkoAAAAJ,548,253,15,10,17,11
54,Yihan Wang,https://scholar.google.com/citations?hl=en&use...,"[international business, economic geography, s...",Ex4TwC8AAAAJ,34,34,3,3,1,1
19,Nabil Khelil,https://scholar.google.com/citations?hl=en&use...,[entrepreneurship - entrepreneurial failure …],DP_OufkAAAAJ,517,398,9,7,8,5


In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, lxml, os

from serpapi import GoogleScholarSearch
from urllib.parse import urlsplit, parse_qsl



def scrape_all_authors_articles(author_id: str):
    params = {
        "user": author_id,   # user-id
        "hl": "en",          # language
        "gl": "us",          # country to search from
        "cstart": 0,         # articles page. 0 is the first page
        "pagesize": "100"    # articles per page
        }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
        }

    all_articles = []

    articles_is_present = True
    while articles_is_present:
        html = requests.post("https://scholar.google.com/citations", params=params, headers=headers, timeout=30)
        soup = BeautifulSoup(html.text, "lxml")

        for index, article in enumerate(soup.select(".gsc_a_tr"), start=1):
            try:
                article_title = article.select_one(".gsc_a_at").text
            except: article_title = None

            try:
                article_link = f'https://scholar.google.com{article.select_one(".gsc_a_at")["href"]}'
            except: article_link = None

            try:
                article_authors = article.select_one(".gsc_a_at+ .gs_gray").text
            except: article_authors = None

            try:
                article_publication = article.select_one(".gs_gray+ .gs_gray").text
            except: article_publication = None
                
            try:
                article_year = article.select_one(".gsc_a_hc").text
            except: article_year = None

            all_articles.append({
                "title": article_title,
                "link": article_link,
                "authors": article_authors,
                "publication": article_publication,
                "article_year": article_year
                })

        # this selector is checking for the .class that contains: "There are no articles in this profile."
        # example link: https://scholar.google.com/citations?user=VjJm3zYAAAAJ&hl=en&cstart=500&pagesize=100
        if soup.select_one(".gsc_a_e"):
            articles_is_present = False
        else:
            params["cstart"] += 100  # paginate to the next page

    pd.DataFrame(data=all_articles).to_csv(f"Articles/google_scholar_{params['user']}_articles.csv", encoding="utf-8", index=False)

user_ids = data['author_ID'].tolist()

for _id in user_ids:
    scrape_all_authors_articles(author_id=_id)
    
    
# # SerpApi solution

# def serpapi_scrape_all_author_articles(author_id: str):
#     params = {
#         "api_key": os.getenv("API_KEY"),    # your SerpApi API key
#         "engine": "google_scholar_author",  # search engine
#         "hl": "en",                         # language
#         "author_id": author_id,             # author ID
#         "start": "0",                       # articles page
#         "num": "100"                        # articles per page
#         }

#     search = GoogleScholarSearch(params)    # where data extraction happens on SerpApi backend.

#     all_articles = []

#     articles_is_present = True
#     while articles_is_present:
#         results = search.get_dict()         # JSON -> Python dictionary

#         for index, article in enumerate(results["articles"], start=1):
#             title = article.get("title")
#             link = article.get("link")
#             authors = article.get("authors")
#             publication = article.get("publication")
#             citation_id = article.get("citation_id")
#             year = article.get("year")

#             print(title)

#             all_articles.append({
#                 "title": title,
#                 "link": link,
#                 "authors": authors,
#                 "publication": publication,
#                 "citation_id": citation_id,
#                 "year": year
#                 })

#         if "next" in results.get("serpapi_pagination", []):
#             # split URL in parts as a dict() and update "search" variable to a new page
#             search.params_dict.update(dict(parse_qsl(urlsplit(results["serpapi_pagination"]["next"]).query)))
#         else:
#             articles_is_present = False

#     pd.DataFrame(data=all_articles).to_csv(f"serpapi_google_scholar_{params['author_id']}_articles.csv", encoding="utf-8", index=False)


# serpapi_scrape_all_author_articles(author_id="VjJm3zYAAAAJ")

In [13]:
## read the tables 
import os 
files= [file for file in os.listdir('Articles/') if file.endswith('.csv')]

df = pd.DataFrame()

for file in files:
    read = pd.read_csv('Articles'+'/'+file)
    print(file)
    read['author_ID'] = file.split('_')[2]
    df = df.append(read)
    
df.head()

google_scholar_2T8gU8AAAAAJ_articles.csv


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append

google_scholar_4KLCJVgAAAAJ_articles.csv
google_scholar_4V2lG70AAAAJ_articles.csv
google_scholar_4ZCoTcUAAAAJ_articles.csv
google_scholar_7MtUDEwAAAAJ_articles.csv
google_scholar_8R7Cg9AAAAAJ_articles.csv
google_scholar_9ZyidVQAAAAJ_articles.csv
google_scholar_BcTSDckAAAAJ_articles.csv
google_scholar_bHUDVG8AAAAJ_articles.csv
google_scholar_bMvX7x4AAAAJ_articles.csv
google_scholar_b_DFLQQAAAAJ_articles.csv
google_scholar_C6oJtcgAAAAJ_articles.csv
google_scholar_cnlixw0AAAAJ_articles.csv


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append

google_scholar_cqyUcDoAAAAJ_articles.csv
google_scholar_D0slPrsAAAAJ_articles.csv
google_scholar_Db4aA10AAAAJ_articles.csv
google_scholar_DgrxQGwAAAAJ_articles.csv
google_scholar_DP_OufkAAAAJ_articles.csv
google_scholar_DtnlwoAAAAAJ_articles.csv
google_scholar_Ex4TwC8AAAAJ_articles.csv
google_scholar_f-nuL0sAAAAJ_articles.csv
google_scholar_FsTjmgUAAAAJ_articles.csv
google_scholar_fz4zTlEAAAAJ_articles.csv
google_scholar_g7lQUh8AAAAJ_articles.csv
google_scholar_g9DEiLEAAAAJ_articles.csv


...

google_scholar_R80_WFUAAAAJ_articles.csv
google_scholar_RfEoWAgAAAAJ_articles.csv
google_scholar_rN_7AkoAAAAJ_articles.csv
google_scholar_TkTj0I4AAAAJ_articles.csv
google_scholar_TpTmMrkAAAAJ_articles.csv
google_scholar_TqBwgC4AAAAJ_articles.csv
google_scholar_TSXET9gAAAAJ_articles.csv
google_scholar_Twj3qDQAAAAJ_articles.csv
google_scholar_vg4TNUkAAAAJ_articles.csv
google_scholar_vNx4Sz8AAAAJ_articles.csv
google_scholar_x-RZHpkAAAAJ_articles.csv
google_scholar_xaceBrYAAAAJ_articles.csv
google_scholar_Xf2BqssAAAAJ_articles.csv
google_scholar_xglEZBkAAAAJ_articles.csv
google_scholar_XmiJS4AAAAAJ_articles.csv
google_scholar_xsoh9wcAAAAJ_articles.csv
google_scholar_YCSIH30AAAAJ_articles.csv
google_scholar_YseQa5wAAAAJ_articles.csv
google_scholar_yxk7n1kAAAAJ_articles.csv
google_scholar_zuUqL2IAAAAJ_articles.csv
google_scholar_zZG_s_8AAAAJ_articles.csv


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append

google_scholar__fjAt0QAAAAJ_articles.csv
google_scholar__TyiCq4AAAAJ_articles.csv


C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_7084\1980432664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(read)


,title,link,authors,publication,article_year,author_ID
0,Economic policy uncertainty and stock markets:...,https://scholar.google.com/citations?view_op=v...,"M Arouri, C Estay, C Rault, D Roubaud","Finance Research Letters 18, 136-141, 2016",2016.0,2T8gU8AAAAAJ
1,Entrepreneurship: From motivation to start-up,https://scholar.google.com/citations?view_op=v...,"C Estay, F Durrieu, M Akhter",Journal of international Entrepreneurship 11 (...,2013.0,2T8gU8AAAAAJ
2,How and when compulsory citizenship behavior l...,https://scholar.google.com/citations?view_op=v...,"P He, Z Peng, H Zhao, C Estay","Journal of Business Ethics 155 (1), 259-274, 2019",2019.0,2T8gU8AAAAAJ
3,Compulsory citizenship behavior and employee s...,https://scholar.google.com/citations?view_op=v...,"P He, X Wang, M Wu, C Estay",Social Behavior and Personality: an internatio...,2018.0,2T8gU8AAAAAJ
4,How destructive leadership influences compulso...,https://scholar.google.com/citations?view_op=v...,"M Wu, Z Peng, C Estay","Chinese Management Studies, 2018",2018.0,2T8gU8AAAAAJ


In [14]:
dataframe = pd.merge(output, df, on = 'author_ID')

In [15]:
dataframe.drop('link', axis = 1, inplace = True)
# dataframe['article_year'].astype(int)
dataframe

,profile_name,profile_link,profile_interests,author_ID,Total citations,Citation l 5 yrs,Total H-index,H-index l 5 yrs,Total i10-index,i10-index l 5 yrs,title,authors,publication,article_year
0,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102,Selecting green suppliers based on GSCM practi...,"D Kannan, ABL de Sousa Jabbour, CJC Jabbour",European Journal of operational research 233 (...,2014.0
1,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102,Industry 4.0 and the circular economy: a propo...,"ABL de Sousa Jabbour, CJC Jabbour, M Godinho F...","Annals of Operations Research 270 (1), 273-286...",2018.0
2,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102,When titans meet–Can industry 4.0 revolutionis...,"ABL de Sousa Jabbour, CJC Jabbour, C Foropon, ...",Technological Forecasting and Social Change 13...,2018.0
3,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102,Green human resource management and green supp...,"CJC Jabbour, ABL de Sousa Jabbour","Journal of Cleaner Production 112, 1824-1833, ...",2016.0
4,Ana Beatriz Lopes de Sousa Jabbour,https://scholar.google.com/citations?hl=en&use...,"[Sustainable Supply Chain Management, Circular...",oOmOcaAAAAAJ,9808,8650,49,44,109,102,Environmental management and operational perfo...,"CJC Jabbour, ABL de Sousa Jabbour, K Govindan,...","Journal of Cleaner Production 47, 129-140, 2013",2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3363,Aneta Hamza-Orlinska,https://scholar.google.com/citations?hl=en&use...,[Human Resource Management],C6oJtcgAAAAJ,1,1,1,1,0,0,Corporate Commitment to Diversity in the Local...,AA HAmzA-OrlinskA,Journal of Corporate Responsibility and Leader...,2017.0
3364,Aneta Hamza-Orlinska,https://scholar.google.com/citations?hl=en&use...,[Human Resource Management],C6oJtcgAAAAJ,1,1,1,1,0,0,Corporate Commitment to Diversity in the Local...,A Orlinska,Central and Eastern Europe in the Changing Bus...,2017.0
3365,Aneta Hamza-Orlinska,https://scholar.google.com/citations?hl=en&use...,[Human Resource Management],C6oJtcgAAAAJ,1,1,1,1,0,0,NaN,NaN,NaN,NaN
3366,Eléonore Mandel,https://scholar.google.com/citations?hl=en&use...,[Intercultural management - Intercultural …],bMvX7x4AAAAJ,1,1,1,1,0,0,The French culturalist way: an interpretative ...,E Mandel,"M@ n@ gement 15 (4), 441-451, 2012",2012.0


In [16]:
dataframe.to_excel('../Output/dataframe.xlsx', index = False)

# 2. ResearchGate 

In [26]:
from parsel import Selector
from playwright.sync_api import sync_playwright
import re, json, time


def scrape_institution_members(institution: str):
    with sync_playwright() as p:
        
        institution_memebers = []
        page_num = 1
        
        members_is_present = True
        while members_is_present:
            
            browser = p.chromium.launch(headless=True, slow_mo=50)
            page = browser.new_page()
            page.goto(f"https://www.researchgate.net/institution/{institution}/members/{page_num}")
            selector = Selector(text=page.content())
            
            print(f"page number: {page_num}")
            
            for member in selector.css(".nova-legacy-v-person-list-item"):
                name = member.css(".nova-legacy-v-person-list-item__align-content a::text").get()
                link = f'https://www.researchgate.net{member.css(".nova-legacy-v-person-list-item__align-content a::attr(href)").get()}'
                profile_photo = member.css(".nova-legacy-l-flex__item img::attr(src)").get()
                department = member.css(".nova-legacy-v-person-list-item__stack-item:nth-child(2) span::text").get()
                desciplines = member.css("span .nova-legacy-e-link::text").getall()
                
                institution_memebers.append({
                    "name": name,
                    "link": link,
                    "profile_photo": profile_photo,
                    "department": department,
                    "descipline": desciplines
                })
                
            # check for Page not found selector
            if selector.css(".headline::text").get():
                members_is_present = False
            else:
                time.sleep(2) # use proxies and captcha solver instead of this
                page_num += 1 # increment a one. Pagination

        print(json.dumps(institution_memebers, indent=2, ensure_ascii=False))
        print(len(institution_memebers)) # 624 from a EM-Normandie-Business-School

        browser.close()
        
        """
        you can also render the page and extract data from the inline JSON string,
        however, it's messy and from my perspective it is easier to scrape the page directly.
        """
        
        # https://regex101.com/r/8qjfnH/1
#         extracted_data = re.findall(r"\s+RGCommons\.react\.mountWidgetTree\(({\"data\":{\"menu\".*:true})\);;",
#                                str(page.content()))[0]
#         json_data = json.loads(extracted_data)
#         print(json_data)
    
scrape_institution_members(institution="EM-Normandie-Business-School")

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

# 3. EDA

In [ ]:
ot = {}

for i in data['author_ID'].tolist():
    s = scrape_all_authors_articles(author_id=i)
    titles = [" - ".join(s['title'])]
    ot[i] = titles
ot = pd.DataFrame(ot).T.reset_index()
ot.columns = ['author_ID', 'Titles']
ot

In [ ]:
data = pd.merge(data, ot, on = 'author_ID')

In [ ]:
data.to_excel('../Output/Dataset.xlsx')

In [ ]:
from wordcloud import WordCloud

In [ ]:
## clean the coprus 
import nltk
from nltk.corpus import stopwords
print(stopwords.words('french'))

In [ ]:
from nltk.tokenize import word_tokenize
data['Titles'] = data['Titles'].apply(word_tokenize)

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english') + stopwords.words('french'))
data['Titles']= data['Titles'].apply(lambda x:[word for word in x if word not in stop_words])

In [ ]:
m = " ".join(data['Titles'].iloc[0])

In [ ]:
word_cloud = WordCloud(width=1200, height=800, collocations = False, background_color = 'white').generate(m)
plt.imshow(word_cloud, interpolation='bilinear', cmap = 'Reds')
plt.axis("off")
plt.title('EM Normandie Business School wordcould', loc = 'left')
plt.tight_layout(pad=0)
plt.savefig('EM.png', dpi = 400)
# plt.savefig(f'../WordCould/{i}.png', dpi = 400, facecolor = 'w')

In [ ]:
## Display the generated Word Cloud

for i in data['profile_name']:
    d = " ".join(data[data['profile_name'] == i].Titles.iloc[0])
    word_cloud = WordCloud(width=1200, height=800, collocations = False, background_color = 'white').generate(d)
    plt.imshow(word_cloud, interpolation='bilinear', cmap = 'Reds')
    plt.axis("off")
    plt.title(f'Author: {i}', loc = 'left')
    plt.tight_layout(pad=0)
    plt.savefig(f'../WordCould/{i}.png', dpi = 400, facecolor = 'w')
    # plt.show()